In [5]:
import pandas as pd
import matplotlib
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.tree import DecisionTreeClassifier

In [6]:
df_constructors = pd.read_csv('f1data/constructors.csv')
df_drivers = pd.read_csv('f1data/drivers.csv')
raceID = pd.read_csv('f1data/races.csv')
df_results = pd.read_csv('f1data/results.csv')

In [12]:
print(df_results.columns)

# Extract only relevant information about the race for training purposes
df_race = raceID[["raceId", "year", "round", "circuitId"]].copy()

Index(['resultId', 'raceId', 'driverId', 'constructorId', 'number', 'grid',
       'position', 'positionText', 'positionOrder', 'points', 'laps', 'time',
       'milliseconds', 'fastestLap', 'rank', 'fastestLapTime',
       'fastestLapSpeed', 'statusId'],
      dtype='object')


Reasons to analyze data from 2014 and onwards:
    1. Modern era relevance: 2014 marked the beginning of the turbo-hybrid era in Formula 1, which brought significant changes to car design, power units, and overall performance. This era is most relevant to current racing conditions.
    2. Technological advancements: The sport has seen rapid technological progress in recent years. Data from before 2014 may not accurately reflect current car capabilities, aerodynamics, or energy recovery systems.
    3. Rule changes: Major regulation changes were implemented in 2014, affecting aerodynamics, fuel consumption, and other aspects of car design. These rules have shaped the current competitive landscape.
    4. Team dynamics: The performance of teams and their hierarchies have shifted significantly since 2014. Using data from this point onwards would provide a more accurate picture of current team strengths and weaknesses.
    5. Driver line-ups: Many current drivers entered F1 around or after 2014. This data set would include most relevant information about current driver performances.
    6. Data quality and quantity: More recent data is likely to be more detailed and comprehensive, offering better insights for prediction modeling.
    7. Sufficient historical context: A decade of data (2014-2024) provides enough historical context to identify trends and patterns without being overly influenced by outdated information.

In [17]:
df_race = df_race.sort_values(by=['year', 'round'])
df_race = df_race[df_race["year"] >= 2014]
print(df_race)

      raceId  year  round  circuitId
897      900  2014      1          1
898      901  2014      2          2
899      902  2014      3          3
900      903  2014      4         17
901      904  2014      5          4
...      ...   ...    ...        ...
1120    1140  2024     20         32
1121    1141  2024     21         18
1122    1142  2024     22         80
1123    1143  2024     23         78
1124    1144  2024     24         24

[228 rows x 4 columns]


In [18]:
df_res = df_results[['raceId', 'driverId', 'constructorId', 'grid', 'positionOrder']].copy()
print(df_res)
duplicates = df_race.duplicated()
num_duplicates = duplicates.sum()
print(f"Number of duplicate rows: {num_duplicates}")
print(df_race)

       raceId  driverId  constructorId  grid  positionOrder
0          18         1              1     1              1
1          18         2              2     5              2
2          18         3              3     7              3
3          18         4              4    11              4
4          18         5              1     3              5
...       ...       ...            ...   ...            ...
26514    1132       839            214    18             16
26515    1132       815              9     0             17
26516    1132       855             15    14             18
26517    1132       847            131     1             19
26518    1132       842            214    19             20

[26519 rows x 5 columns]
Number of duplicate rows: 0
      raceId  year  round  circuitId
897      900  2014      1          1
898      901  2014      2          2
899      902  2014      3          3
900      903  2014      4         17
901      904  2014      5          4
... 

In [35]:
def calculate_rolling_average_points(df_results, n_races=244, start_race_id=900):
    # Ensure the dataframe is sorted by raceId
    df_results = df_results.sort_values('raceId')
    
    # Filter for races from 2014 onwards
    df_results = df_results[df_results['raceId'] >= start_race_id]
    
    # Group by driver and sort by raceId
    grouped = df_results.groupby('driverId')
    
    # Function to calculate rolling average for a group
    def rolling_avg(group):
        return group['points'].rolling(window=n_races, min_periods=1).mean()
    
    # Apply rolling average calculation
    df_results['rolling_avg_points'] = grouped.apply(rolling_avg).reset_index(level=0, drop=True)
    
    return df_results


# Calculate rolling average for last 5 races
results_with_rolling_avg = calculate_rolling_average_points(df_results, n_races=5)

# Display the results
print(results_with_rolling_avg[['raceId', 'driverId', 'points', 'rolling_avg_points']])

       raceId  driverId  points  rolling_avg_points
22146     900        13     0.0                 0.0
22137     900        16     0.0                 0.0
22143     900       828     0.0                 0.0
22145     900         1     0.0                 0.0
22142     900       813     0.0                 0.0
...       ...       ...     ...                 ...
26518    1132       842     0.0                 1.2
26517    1132       847     0.0                12.4
26515    1132       815     0.0                 2.0
26511    1132       817     0.0                 1.2
26508    1132       852     1.0                 1.0

[4386 rows x 4 columns]


C:\Users\alexa\AppData\Local\Temp\ipykernel_9084\4217960290.py:16: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_results['rolling_avg_points'] = grouped.apply(rolling_avg).reset_index(level=0, drop=True)


,resultId,raceId,driverId,constructorId,number,grid,position,positionText,positionOrder,points,laps,time,milliseconds,fastestLap,rank,fastestLapTime,fastestLapSpeed,statusId,rolling_avg_points
7572,7573,1,1,1,1,18,\N,D,20,0.0,58,\N,\N,39,13,1:29.020,214.455,2,0.000000
7579,7580,2,1,1,1,12,7,7,7,1.0,31,+1:00.733,4312825,17,11,1:39.141,201.276,1,0.500000
7598,7599,3,1,1,1,9,6,6,6,3.0,56,+1:11.866,7135351,39,12,1:54.665,171.138,1,1.333333
7616,7617,4,1,1,1,5,4,4,4,5.0,57,+22.096,5530278,13,6,1:34.915,205.269,1,2.250000
7641,7642,5,1,1,1,14,9,9,9,0.0,65,\N,\N,29,9,1:23.839,199.883,11,1.800000


In [45]:
import pandas as pd
import numpy as np

def calculate_relative_position_improvement(df, driver_id_col, race_id_col, start_position_col, finish_position_col, n_races=None):
    """
    Calculate the average position improvement (starting position - finishing position)
    for each driver over the last N races.
    
    Parameters:
    df (pd.DataFrame): DataFrame containing race results
    driver_id_col (str): Name of the column containing driver IDs
    race_id_col (str): Name of the column containing race IDs
    start_position_col (str): Name of the column containing starting positions
    finish_position_col (str): Name of the column containing finishing positions
    n_races (int, optional): Number of recent races to consider. If None, uses all races.
    
    Returns:
    pd.DataFrame: Original DataFrame with an additional column for average position improvement
    """
    
    # Create a copy of the DataFrame to avoid modifying the original
    df = df.copy()
    
    # Sort the DataFrame by driver and race ID
    df = df.sort_values([driver_id_col, race_id_col])
    
    # Ensure start and finish positions are numeric
    df[start_position_col] = pd.to_numeric(df[start_position_col], errors='coerce')
    df[finish_position_col] = pd.to_numeric(df[finish_position_col], errors='coerce')
    
    # Calculate position improvement for each race
    df['position_improvement'] = df[start_position_col] - df[finish_position_col]
    
    # Function to calculate average improvement
    def avg_improvement(group):
        if n_races is None:
            return group.mean()
        else:
            return group.tail(n_races).mean()
    
    # Calculate average improvement for each driver
    avg_improvements = df.groupby(driver_id_col)['position_improvement'].transform(avg_improvement)
    
    # Add the result to the DataFrame
    df['avg_position_improvement'] = avg_improvements
    
    return df

n_races = 5  # Calculate average over the last 5 races, or use None for all races

df_results = calculate_relative_position_improvement(df_results, 'driverId', 'raceId', 'grid', 'position', n_races)
print(df_results.head())

      resultId  raceId  driverId  constructorId number  grid  position  \
7572      7573       1         1              1      1    18       NaN   
7579      7580       2         1              1      1    12       7.0   
7598      7599       3         1              1      1     9       6.0   
7616      7617       4         1              1      1     5       4.0   
7641      7642       5         1              1      1    14       9.0   

     positionText  positionOrder  points  ...       time milliseconds  \
7572            D             20     0.0  ...         \N           \N   
7579            7              7     1.0  ...  +1:00.733      4312825   
7598            6              6     3.0  ...  +1:11.866      7135351   
7616            4              4     5.0  ...    +22.096      5530278   
7641            9              9     0.0  ...         \N           \N   

     fastestLap rank fastestLapTime fastestLapSpeed statusId  \
7572         39   13       1:29.020         214.455 